In [1]:
# plot of liquity 
import scipy 
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.widgets import Slider
import seaborn as sns
import numpy as np 
import random
import pandas as pd 
from ipywidgets import *
from scipy.stats import norm
random.seed(1)
set_random_seed = True
t =  np.linspace(91,0,92)/365
Real_vol = 0.5
Growth_rate =1
LTV0 = 0.8
bs_vol = 0.5
y_price = 50000
Collateral = 10000000
pool_init_x = 5000000
pool_init_y = 100
pool_init_wx = 0.5


In [2]:
class weights_generator: 
    """
        Purpose: generate weights based on different strategies 
                1.  Black-shoals option pricing model and observed price. 
                    A delta replicating strategy of a call option on Token / Collateral. 
                2.  Exp. smooth of 1:  0.95 * w(t-1) +  0.05 * w(t)
                3.  Moving average of 1
        
        Assumption: 
            1. Rebalnce following the above strategies.  
            2. Price percentage change (approximately equals to log(p(t+1)/p(t))) follows a linear growth treand 
               (slope = growth rate) plus a random walk Brownie motion. 
                              
        Input: (attributes) 
            - t: term of the loan 
            - y_price: initial price y token (e.g. 1 btc = 50000 btc) 
            - bs_vol: black shoral option volatilty 
            - Growth_rate: linear growth slope of y price 
            - Real_vol: growth Brownie motion volatility 
            
            Additional input:  
            
                - ma_window: weights moving average window.   
                         Pad 0.5 if t < ma_window 
                - expotential smothing weight: 0.95 
       
        Methods: 
            - BS_weights(*args) 
                - "BS"
                - 'MA', ma_window 
                - 'SM', wt 
    
        Output: 
            - A dataframe with columns: ['t', 'y_price', 'weights_ytoken', 'slippage_rebalance'] 
    
    """
    def __init__(self, t, y_price0, bs_vol, Growth_rate, Real_vol):
        self.t = t
        self.y_price0 = y_price0
        self.bs_vol = bs_vol 
        self.Growth_rate = Growth_rate
        self.Real_vol = Real_vol 

    def BS_weights(self, *args): 
        if set_random_seed == True:
            np.random.seed(101)
       # loan life duation between t(i) - t(i+1)
        delta_t = self.t[:-1]-self.t[1:]
        # token price percent change following a linear growth curve plus a random walk with volatility = Real_vol 
        rw = np.random.random(size = len(self.t)-1)
        delta_price = delta_t * (self.Growth_rate - 0.5*self.Real_vol**2) +  self.Real_vol *np.sqrt(delta_t)*norm.ppf(rw)

        # calcualte price with exponetial of cumulative sum of log price differences.  
        price_logdiff= np.hstack([0, delta_price]).cumsum()
        price = self.y_price0*np.exp(price_logdiff)
        # reblance weights by black scholes option equestion. 
        w_y = norm.cdf((price_logdiff[:-1] + self.t[:-1]*(0.5*self.bs_vol**2))/self.bs_vol/np.sqrt(self.t[:-1]))
        # set a boundary [1e-8, 1-1e-8] to avoid 0 as denominator 
        w_y[w_y< 1e-8] = 1e-8   
        w_y[w_y> (1- 1e-8)] = 1- 1e-8
        weights_ytoken = np.append(w_y, w_y[-1])
        args = list(np.array(args).flat)
        #print(args)
        if args[0] == 'BS':
            _wt = weights_ytoken
        elif args[0]== 'MA':
            _wt = self.moving_avg(weights_ytoken, args[1])
        elif args[0] == 'SM':
            _wt = self.sm(weights_ytoken, float(args[1]))
        elif args[0]== '50/50':
            _wt = np.array([0.5]*len(weights_ytoken))
        else:
            print('no such smooth method')
        slippage = np.append([0], (_wt[:-1]/_wt[1:])**_wt[1:]*( (1-_wt[:-1])/(1-_wt[1:]))**(1-_wt[1:])-1)
        # formate the output file as a dataframe for downstream analysis 
        _temp= np.column_stack((t, price, _wt, slippage))
        out_vec = pd.DataFrame(_temp, columns = ['t', 'y_price', 'weights_ytoken', 'slippage_rebalance'])
        return out_vec 

    def moving_avg(self, wt, ma_window):
        ma_window = int(ma_window)
        assert isinstance(ma_window, int), 'Moving average window must be integer!' 
        # pad the weights by initial weights 
        _w_padding = np.append(np.array([wt[0]]*(ma_window-1)), wt)
        _wt_ma = np.convolve(_w_padding, np.ones(ma_window)/ma_window, mode = 'valid')
        return _wt_ma

    def sm(self, wt, exp_w): 
        assert 0<exp_w<1, 'smoothing weight must be between 0 and 1!'
        _wt_sm = []
        _w_padding = np.append(np.array(wt[0]), wt)
        _newwt = wt[0]
        for i in range(len(wt)):
            _newwt = _newwt*exp_w + _w_padding[i+1]*(1-exp_w)
            _wt_sm.append(_newwt) 
        return np.array(_wt_sm)
    


In [ ]:
def gen_data_vec(w_btc, price, slippage, V0, rebate=0, fee_rate=0):
    
    """
        given weight, price, V0 generate 
        - x: btc size 
        - y: usd size 
        - V: invariance function 
        - G: portfolio value 
    """
    w_y = w_btc.values
    w_x = 1-w_y
    p = price.values 
    fee = np.abs(w_btc.diff(1).fillna(0).values)*fee_rate
    slip = slippage.values
    # Update invariance function V by factor 
    _factor = (w_y[:-1]/(w_x[:-1]*p[:-1]))**(w_x[:-1]*w_y[1:]) * (w_x[:-1]*p[:-1]/(w_y[:-1]))**(w_y[:-1]*w_x[1:])
    # add rebate and fee to update factor
    _rebate = -slip*(rebate) + fee
    _factor += _rebate[1:]
    V = np.hstack([V0, V0*_factor.cumprod()])
    y = V*((w_y/(w_x*p))**w_x)
    x = V*((w_x*p)/w_y)**w_y
    g = x + y*p 
    _rebate = -slip*(rebate) + fee
    #g_rebate= np.append(g[0], g[1:] + g[:-1]*(-slip[1:]*(rebate) + fee[1:]))
    
    df = pd.DataFrame(np.column_stack((x,y,V,g,_rebate)), columns = 
                      ['x_locked', 'y_locked','V','coll_with_rebate', 'rebate'])
    return df



In [ ]:
# get_episode_full(t,y_price_init, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
#                      pool_init_x, pool_init_y, pool_init_wx, 'SM', 0.1)

In [ ]:
def get_episode_full(t,y_price_init, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate_rate,
                     pool_init_x, pool_init_y, pool_init_wx, *args):
    
    """
    Purpuse: 
        generate one simulation based on selected parameters and random price change following the Browian motion.  
        pool attributes are rescorded at each step. 
        Rebate and fee values are back to the pool. 
    
    Input: 
        - t: term of the loan (e.g. 90 days)
        - y_price: initial price of loan token (e.g. USDC = 3e-5 btc) 
        - bs_vol: black shoral option volatilty 
        - Growth_rate: linear growth slope of y price 
        - Real_vol: growth Brownie motion volatility 
        - Collateral amount (e.g. 200 BTC) 
        - LTV0: Initial LTV (e.g. 90%)
        - fee: e.g. 0.15% of absolute weight changes 
        - rebate: e.g. 50% slippage due to rebalance weigths 
        - pool_init_x: Initial x size 
        - pool_init_y: Initial y size 
        - Pool_init_wx: Inital x weight (e.g. 50%)
        
        Additional input: 
            - ('BS') Strickly follow BS
            - ('MA', 7) N: moving average window, padding 0.5 if t < N 
            - ('SM', 0.95) expotential smothing weight: 0.95 
    
    Output: 
        Dataframe with the following key measures. 
    
        - 1. Token price at time t 
        - 2. Reblanced weight at time t 
        - 3. Daily pool value loss (relative to collaterals)
                slippage due to rebalance weights 
        - 4. Collateral value 
        - 5. Collateral value with rebate 
        - 6. LTV relative to collateral
        - 7. LTV relative to collateral with rebate   

    """
    # call weights_generator class to simulate various weights rebalnce strategies  
    #print(args)
    wt_gen = weights_generator(t, y_price, bs_vol, Growth_rate, Real_vol)
    _out = wt_gen.BS_weights(args)
    w_btc = _out.weights_ytoken
    price = _out.y_price 
    V0 = pool_init_y**w_btc[0]*pool_init_x**(1-w_btc[0]) 
    borrowed_usd = Collateral*LTV0
    slippage = _out.slippage_rebalance
    # calculate pool values 
    episode = gen_data_vec(w_btc, price, slippage, V0, rebate_rate, fee_rate)
    episode_full = pd.concat([_out, episode], axis=1)
    episode_full['ltv_with_rebate'] = borrowed_usd/episode_full['coll_with_rebate']    
    episode_full['wt_chg'] = episode_full['weights_ytoken'].diff(1).fillna(0)
    episode_full['pvtc_rebate'] = episode_full['coll_with_rebate']/episode_full['y_price']/(Collateral/y_price_init)
    return episode_full


In [ ]:
def episode_plot(var_list, *args, set_random_seed = True, row=2, col=3, Real_vol = 0.5, Growth_rate = 0, LTV0 = 0.8, bs_vol = 0.5,
    y_price_init = 50000, Collateral = 10000000, pool_init_x = 5000000, pool_init_y = 100, pool_init_wx = 0.5,
    fee_rate = 0.0015,  rebate=0.5 ):
    %matplotlib notebook
    import matplotlib.pyplot as plt 
    fig, axs = plt.subplots(row,col,figsize=(8,6))
    axs[-1, -1].axis('off')
    _args = list(np.array(args).flat)
    # Create axes for sliders
    ax_growth = fig.add_axes([0.73, 0.3, 0.2, 0.02])
    ax_growth.spines['top'].set_visible(True)
    ax_growth.spines['right'].set_visible(True)

    ax_real_vol = fig.add_axes([0.73, 0.26, 0.2, 0.02])
    ax_real_vol.spines['top'].set_visible(True)
    ax_real_vol.spines['right'].set_visible(True)

    ax_ltv = fig.add_axes([0.73, 0.22, 0.2, 0.02])
    ax_ltv.spines['top'].set_visible(True)

    ax_bs_vol = fig.add_axes([0.73, 0.18, 0.2, 0.02])
    ax_bs_vol.spines['top'].set_visible(True)
    ax_bs_vol.spines['right'].set_visible(True)

    ax_rebate = fig.add_axes([0.73, 0.14, 0.2, 0.02])
    ax_rebate.spines['top'].set_visible(True)
    ax_rebate.spines['right'].set_visible(True)
    
    if _args[0] == 'MA': 
        ax_ma = fig.add_axes([0.73, 0.10, 0.2, 0.02])
        ax_ma.spines['top'].set_visible(True)
        ax_ma.spines['right'].set_visible(True)
    elif _args[0] == 'SM': 
        ax_sm = fig.add_axes([0.73, 0.10, 0.2, 0.02])
        ax_sm.spines['top'].set_visible(True)
        ax_sm.spines['right'].set_visible(True)
    
    ax_fee = fig.add_axes([0.73, 0.34, 0.2, 0.02])
    ax_fee.spines['top'].set_visible(True)
    ax_fee.spines['right'].set_visible(True)

    # Create sliders
    s_real_vol= Slider(ax=ax_real_vol, label='Real Vol', valmin=0.1, valmax=1.0, valinit=0.5, valfmt=' %1.2f ', facecolor='#cc7000')
    s_growth = Slider(ax=ax_growth, label='Growth', valmin=-2, valmax=2, valinit=0, valfmt=' %1.2f', facecolor='#cc7000')
    s_bs_vol= Slider(ax=ax_bs_vol, label='BS Vol', valmin=0.1, valmax=1.0, valinit=0.5, valfmt=' %1.2f ', facecolor='#cc7000')
    s_ltv = Slider(ax=ax_ltv, label='LTV0', valmin=0.7, valmax=0.95, valinit=0.9, valfmt=' %1.2f', facecolor='#cc7000')
    s_rebate = Slider(ax=ax_rebate, label='Rebate', valmin=0.0, valmax=1, valinit=0.5, valfmt=' %1.2f', facecolor='#cc7000')
    s_fee = Slider(ax=ax_fee, label='Fee', valmin=0.0, valmax=0.01, valinit=0.0015, valfmt=' %1.4f', facecolor='#cc7000')
    if _args[0] == 'MA':
        s_ma =     Slider(ax=ax_ma, label='MA', valmin=1, valmax=30, valinit=7, valfmt=' %0.0f ', facecolor='#cc7000')
    elif _args[0] == 'SM':
        s_sm =  Slider(ax=ax_sm, label='SM', valmin=0, valmax=1, valinit=0.95, valfmt=' %1.2f ', facecolor='#cc7000')
    
    # Plot default data
    #t =  np.linspace(91,0,92)/365
    _data = get_episode_full(t,y_price_init, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                     pool_init_x, pool_init_y, pool_init_wx, _args)
  
    x = _data.index.values
    y1 = _data[var_list[0]]/y_price_init
    y2a = _data[var_list[1]]
    y2b = 1-_data[var_list[1]]    
    y3 = _data[var_list[2]]
    y4 = _data[var_list[3]]
    y5 = _data[var_list[4]]
    y6 = _data[var_list[5]]

    f_d1, = axs[0,0].plot(x, y1, linewidth=2.5)
    f_d2a, = axs[0,1].plot(x, y2a, linewidth=2.5)
    f_d2b, = axs[0,1].plot(x, y2b, linewidth=2.5)
    f_d3, = axs[0,2].plot(x, y3, linewidth=2.5)
    f_d4, = axs[1,0].plot(x, y4, linewidth=2.5)
    f_d5, = axs[1,1].plot(y6, y5, marker= 'o', linestyle = 'None', linewidth=1)
    

    axs[0,0].set_ylim(0.5,2.5)
    axs[0,0].axhline(y = 1, color = 'r', linestyle = '--')
    axs[0,1].set_ylim(-0.02,1.02)
    axs[0,1].axhline(y = 0.5, color = 'r', linestyle = '--')
    axs[0,2].set_ylim(0.5,2)
    axs[0,2].axhline(y = 1, color = 'r', linestyle = '--')
    axs[1,0].set_ylim(0.5,2)
    axs[1,0].axhline(y = 1, color = 'r', linestyle = '--')
    axs[1,1].set_ylim(-0.2,0.05)
    axs[1,1].set_xlim(-0.5,0.5)
    axs[1,1].axhline(y = 0, color = 'r', linestyle = '--')
   
    
    axs[0,0].set_ylabel('BTC relative change ')
    axs[0,1].set_ylabel('Weights (USDC: Orange, BTC: Blue)')
    axs[0,2].set_ylabel('LTV-with rebate')
    axs[1,0].set_ylabel('PVTC with rebate') 
    axs[1,1].set_ylabel('Impermanent Loss vs weight chg') 

    for i in range(2):
        for j in range(3):
            axs[i,j].set_xlabel('Day')
            axs[i,j].set_title('({})'.format(3*i+j+1))
    axs[1,1].set_xlabel('weight changes')
    # Update values
    def update(val):
        _real_vol = s_real_vol.val
        _growth = s_growth.val
        _bs_vol = s_bs_vol.val
        _ltv = s_ltv.val
        _rebate = s_rebate.val
        _fee = s_fee.val
        if _args[0] == 'MA':
            _ma = int(s_ma.val)
        elif _args[0] == 'SM':
            _sm = s_sm.val

        if _args[0] == 'MA':
            _data = get_episode_full(t,y_price_init, _bs_vol, _growth, _real_vol, Collateral, _ltv, _fee, _rebate,
                     pool_init_x, pool_init_y, pool_init_wx, _args[0], _ma)
        elif _args[0] == 'SM':
            _data = get_episode_full(t,y_price_init, _bs_vol, _growth, _real_vol, Collateral, _ltv, _fee, _rebate,
                     pool_init_x, pool_init_y, pool_init_wx, _args[0], _sm)
        else:
            _data = get_episode_full(t,y_price_init, _bs_vol, _growth, _real_vol, Collateral, _ltv, _fee, _rebate,
                     pool_init_x, pool_init_y, pool_init_wx, _args[0])
        x = _data.index.values
        _y1 = _data[var_list[0]]/y_price_init
        _y2a = _data[var_list[1]]
        _y2b = 1-_data[var_list[1]]    
        _y3 = _data[var_list[2]]
        _y4 = _data[var_list[3]]
        _y5 = _data[var_list[4]]
        _y6 = _data[var_list[5]]

        f_d1.set_data(x,_y1)
        f_d2a.set_data(x,_y2a)
        f_d2b.set_data(x,_y2b)
        f_d3.set_data(x,_y3)
        f_d4.set_data(x,_y4)
        f_d5.set_data(_y6,_y5)
        fig.canvas.draw_idle()

    s_real_vol.on_changed(update)
    s_growth.on_changed(update)
    s_ltv.on_changed(update)
    s_bs_vol.on_changed(update)
    s_rebate.on_changed(update)
    if _args[0] == 'MA':
        s_ma.on_changed(update)
    elif _args[0] == 'SM':
        s_sm.on_changed(update)
    s_fee.on_changed(update)
    fig.tight_layout()
    plt.show()

In [ ]:
#episode_plot(['y_price', 'weights_ytoken', 'ltv_with_rebate', 'pvtc_rebate', 'slippage_rebalance', 'wt_chg'], '50/50')

In [4]:
def ltv_simulation(s, *args ):
    """
        MC simualtion
            check the distribution of final pool value and risk of default 
    """
    out = []
    for k in range(0,s):
        _out = get_episode_full(t,y_price_init, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                      pool_init_x, pool_init_y, pool_init_wx, args)
        out.append([_out.coll_with_rebate.values[-1], _out.pvtc_rebate.values[-1],_out.ltv_with_rebate.values.max()>1])

    return np.array(out)